In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split

train_image_folder = '/content/drive/MyDrive/image'
train_label_folder = '/content/drive/MyDrive/label'
test_image_folder = '/content/drive/MyDrive/test'

import cv2

input_list = []
output_list = []

for dirname, _, filenames in os.walk(train_image_folder):
    for filename in filenames:
        train_file_img = os.path.join(dirname, filename)
        train_label_img = os.path.join(train_label_folder,filename)

        image = cv2.imread(train_file_img,0)
        image = cv2.resize(image, (256, 256))

        image_features = image.astype('float32') / 255.0
        input_list.append(image_features)

        image = cv2.imread(train_label_img,0)
        image = cv2.resize(image, (256, 256))
        image_features = image.astype('float32') / 255.0
        output_list.append(image_features)

In [3]:
input_list = np.expand_dims(input_list, axis=-1)
output_list = np.expand_dims(output_list, axis=-1)


images = input_list
masks = output_list
images = np.array(images)
masks = np.array(masks)

X_train, y_train = images, masks

In [4]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Concatenate
from tensorflow.keras.models import Model

def unet_model(input_size=(256, 256, 1)):
    inputs = Input(input_size)

    # Encoder
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(512, 3, activation='relu', padding='same')(pool3)
    conv4 = Conv2D(512, 3, activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(pool4)
    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(conv5)

    # Decoder
    up6 = UpSampling2D(size=(2, 2))(conv5)
    merge6 = Concatenate()([conv4, up6])
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(merge6)
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(conv6)

    up7 = UpSampling2D(size=(2, 2))(conv6)
    merge7 = Concatenate()([conv3, up7])
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(merge7)
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(conv7)

    up8 = UpSampling2D(size=(2, 2))(conv7)
    merge8 = Concatenate()([conv2, up8])
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(merge8)
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(conv8)

    up9 = UpSampling2D(size=(2, 2))(conv8)
    merge9 = Concatenate()([conv1, up9])
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(merge9)
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(conv9)

    conv10 = Conv2D(1, 1, activation='sigmoid')(conv9)

    model = Model(inputs, conv10)

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

model = unet_model()
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 256, 256, 1)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 256, 256, 64)         640       ['input_1[0][0]']             
                                                                                                  
 conv2d_1 (Conv2D)           (None, 256, 256, 64)         36928     ['conv2d[0][0]']              
                                                                                                  
 max_pooling2d (MaxPooling2  (None, 128, 128, 64)         0         ['conv2d_1[0][0]']            
 D)                                                                                           

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

seed = 24
batch_size = 10

img_data_gen_args = dict(rotation_range=90,
                         width_shift_range=0.3,
                         height_shift_range=0.3,
                         shear_range=0.5,
                         zoom_range=0.3,
                         horizontal_flip=True,
                         vertical_flip=True,
                         fill_mode='reflect')

mask_data_gen_args = dict(rotation_range=90,
                          width_shift_range=0.3,
                          height_shift_range=0.3,
                          shear_range=0.5,
                          zoom_range=0.3,
                          horizontal_flip=True,
                          vertical_flip=True,
                          fill_mode='reflect',
                          preprocessing_function = lambda x: np.where(x>0, 1, 0).astype(x.dtype)
image_data_generator = ImageDataGenerator(**img_data_gen_args)
image_data_generator.fit(X_train, augment=True, seed=seed)

image_generator = image_data_generator.flow(X_train, seed=seed, batch_size=batch_size)

mask_data_generator = ImageDataGenerator(**mask_data_gen_args)
mask_data_generator.fit(y_train, augment=True, seed=seed)
mask_generator = mask_data_generator.flow(y_train, seed=seed, batch_size=batch_size)

def my_image_mask_generator(image_generator, mask_generator):
    train_generator = zip(image_generator, mask_generator)
    for (img, mask) in train_generator:
        yield (img, mask)

my_generator = my_image_mask_generator(image_generator, mask_generator)

steps_per_epoch = 3

history = model.fit(my_generator, steps_per_epoch=steps_per_epoch, epochs=5)


Epoch 1/5
3/3 [==============================] - 264s 82s/step - loss: 42.7922 - accuracy: 0.9018
Epoch 2/5
3/3 [==============================] - 243s 81s/step - loss: 0.4757 - accuracy: 0.8983
Epoch 3/5
3/3 [==============================] - 248s 83s/step - loss: 0.5756 - accuracy: 0.9063
Epoch 4/5
3/3 [==============================] - 244s 82s/step - loss: 0.5485 - accuracy: 0.9026
Epoch 5/5
3/3 [==============================] - 244s 82s/step - loss: 0.4284 - accuracy: 0.9025


In [6]:
test_input = []

for dirname, _, filenames in os.walk(test_image_folder):
    for filename in filenames:
        test_file_img = os.path.join(dirname, filename)

        image = cv2.imread(test_file_img,0)
        image = cv2.resize(image, (256, 256))

        image_features = image.astype('float32') / 255.0
        test_input.append(image_features)

test_input = np.expand_dims(test_input, axis=-1)
Xtest = np.array(test_input)

In [13]:
from PIL import Image
y_pred = model.predict(Xtest,batch_size= 5)

12/12 [==============================] - 142s 12s/step


In [20]:
output_folder = '/content/drive/MyDrive/image pred final'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)
for i, image in enumerate(y_pred):

    image = (image * 255).astype(np.uint8)

    output_path = os.path.join(output_folder, f'predicted_image_{i}.png')
    cv2.imwrite(output_path, image)

In [25]:
import os
import numpy as np
from PIL import Image
from skimage.metrics import adapted_rand_error, contingency_table, variation_of_information

def load_images_from_folder(folder):
    images = []
    for filename in sorted(os.listdir(folder)):
        img_path = os.path.join(folder, filename)
        img = Image.open(img_path)
        img = img.resize((256, 256))
        img = np.array(img)
        images.append(img)
    return images

ground_truth_folder = '/content/drive/MyDrive/test'
predicted_folder = '/content/drive/MyDrive/image pred final'

ground_truth_images = load_images_from_folder(ground_truth_folder)
predicted_images = load_images_from_folder(predicted_folder)

scores = []

for y_true, y_pred in zip(ground_truth_images, predicted_images):
    y_true = y_true / 255
    y_pred = y_pred / 255
from skimage.metrics import structural_similarity as ssim
def compare_images_ssim(y_true, y_pred):
    grayA = y_true.astype(np.uint8)
    grayB = y_pred.astype(np.uint8)


    # Compute SSIM between two images
    score, diff = ssim(grayA, grayB, full=True)
    return score

scores.append(compare_images_ssim(y_true, y_pred))
print(f"Score: {np.mean(scores):.4f}")


Score: 0.3089
